In [38]:
import numpy as np
import json
import glob
from functools import reduce

In [39]:
def load_results(dataset, model, pooling, layer, metric):
    if 'wd' in metric:
        result_dir = f'../results/{dataset}/{model}/{metric}/{pooling}/{layer}'
    else:
        result_dir = f'../results/{dataset}/{model}/{metric}'
    
    result_fs = glob.glob(f'{result_dir}/*.json')
    results = []
    for file in result_fs:
        with open(file, 'r') as f:
            data = json.load(f)
            print(file, len(data))
        results.extend(data)
    for r in results:
        r['metric'] = r['swd'] if 'wd' in metric else r['metric']
    return [i for i in results if i['corruption level'] != 0]

In [169]:
layers = ['fc']
metric = 'wd'
dataset = 'CIFAR-100'
arch = 'resnet50'
logits_results = reduce(lambda x, y: x + y, [load_results(dataset, arch, 'logits', layer, metric) for layer in layers])

../results/CIFAR-100/resnet50/wd/logits/fc/brightness.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/defocus_blur.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/elastic_transform.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/fog.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/frost.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/gaussian_blur.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/gaussian_noise.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/glass_blur.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/impulse_noise.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/jpeg_compression.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/motion_blur.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/pixelate.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/saturate.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/shot_noise.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/snow.json 6
../results/CIFAR-100/resnet50/wd/logits/fc/spatter.json 6
../results/CIFAR-100/re

In [170]:
len(logits_results)

95

In [171]:
import altair as alt
import pandas as pd

In [172]:
alt.Chart(alt.Data(values=logits_results)).mark_point().encode(
    x=alt.X('metric:Q',title='FOT'), 
    y='error:Q',
    color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
    shape='corruption level:O'
).properties(
    width=250,
    height=200
)

alt.Chart(...)

In [132]:
def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

    results['polynomial'] = coeffs.tolist()

    p = np.poly1d(coeffs)

    yhat = p(x)                
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat - ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    results['determination'] = ssreg / sstot

    return results

In [30]:
from scipy.stats import spearmanr
import math

In [31]:
d = [i['metric'] for i in logits_results if not math.isnan(i['metric'])]
e = [i['error'] for i in logits_results if not math.isnan(i['metric'])]
print(f'{metric} r2:', polyfit(d, e)['determination'])

coef, p = spearmanr(d, e)
print(f'{metric} rho:', coef)

projnorm_500 r2: 0.9139362369366588
projnorm_500 rho: 0.9733431807331023
